In [1]:
from Utils import *
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import os.path


In [2]:
coupling_type = '1JHN'
#XY_Data = pd.read_csv(f'{Config.INPUT_XY}/{coupling_type}.csv', index_col=0)
#n_atoms = 11
#train_csv = load_train()
#XY_Data = build_XY(train_csv, coupling_type, n_atoms, False)
XY_Data = pd.read_csv(f'{Config.INPUT_XY}/{coupling_type}.csv', index_col=0)

In [3]:
#XY_Data, _ = train_test_split(XY_Data, test_size=0.9, random_state=42)

In [4]:
#XY_Data = XY_Data[labels]
#XY_Data['atom_2_6'] = 0
#XY_Data[XY_Data.atom_2 == 6].atom_2_6 = 1
#XY_Data.atom_2.unique()
#labels = [col for col in XY_Data if col.startswith('atom_')]
#labels = ['atom_2', 'atom_3']
#bbb = XY_Data[labels]
#XY_Data = pd.get_dummies(XY_Data, columns = labels, prefix = labels, drop_first = True)
#aaa.head()

In [5]:
XY_Data.d_7_3.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: d_7_3, dtype: float64

In [6]:
XY_Data.columns

Index(['atom_2', 'atom_3', 'atom_4', 'atom_5', 'atom_6', 'atom_7', 'atom_8',
       'atom_9', 'atom_10', 'r_x_1', 'r_x_2', 'r_x_3', 'r_x_4', 'r_x_5',
       'r_x_6', 'r_x_7', 'r_x_8', 'r_x_9', 'r_x_10', 'r_y_2', 'r_y_3', 'r_y_4',
       'r_y_5', 'r_y_6', 'r_y_7', 'r_y_8', 'r_y_9', 'r_y_10', 'r_z_3', 'r_z_4',
       'r_z_5', 'r_z_6', 'r_z_7', 'r_z_8', 'r_z_9', 'r_z_10', 'd_2_0', 'd_2_1',
       'd_3_0', 'd_3_1', 'd_3_2', 'd_4_0', 'd_4_1', 'd_4_2', 'd_4_3', 'd_5_0',
       'd_5_1', 'd_5_2', 'd_5_3', 'd_5_4', 'd_6_0', 'd_6_1', 'd_6_2', 'd_6_3',
       'd_6_4', 'd_6_5', 'd_7_0', 'd_7_1', 'd_7_2', 'd_7_3', 'd_7_4', 'd_7_5',
       'd_8_0', 'd_8_1', 'd_8_2', 'd_8_3', 'd_8_4', 'd_8_5', 'd_9_0', 'd_9_1',
       'd_9_2', 'd_9_3', 'd_9_4', 'd_9_5', 'd_10_0', 'd_10_1', 'd_10_2',
       'd_10_3', 'd_10_4', 'd_10_5', 'scalar_coupling_constant'],
      dtype='object')

In [7]:
XY_train, XY_Val =  train_test_split(XY_Data, test_size=0.1, random_state=228)
X_val, y_val = build_x_y_data(XY_Val)    

In [8]:
X_val.describe()

,atom_2,atom_3,atom_4,atom_5,atom_6,atom_7,atom_8,atom_9,atom_10,r_x_1,...,d_9_2,d_9_3,d_9_4,d_9_5,d_10_0,d_10_1,d_10_2,d_10_3,d_10_4,d_10_5
count,4337.000000,4337.000000,4337.000000,4337.000000,4337.000000,4337.000000,4337.000000,4337.000000,4337.000000,4337.000000,...,4337.000000,4337.000000,4337.000000,4337.000000,4337.000000,4337.000000,4337.000000,4337.000000,4337.000000,4337.000000
mean,4.195527,6.020060,4.056491,4.355545,4.483283,4.685497,4.537007,4.119668,3.975098,-0.506390,...,3.271506,2.805672,3.195402,2.993048,4.079259,3.582247,3.514811,3.018712,3.353807,3.152938
std,2.469472,1.017519,2.942529,2.800968,2.719812,2.571084,2.630641,2.777865,2.848792,0.002920,...,1.025852,0.769508,1.087911,1.132733,0.810863,0.749819,1.077227,0.834264,1.159281,1.176589
min,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.543186,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,6.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.508722,...,2.467563,2.257185,2.385250,2.187172,3.544207,3.051775,2.609186,2.382266,2.467291,2.266688
50%,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,-0.506053,...,3.152108,2.669174,3.111838,2.956278,4.020514,3.519808,3.471629,2.827435,3.261848,3.095590
75%,6.000000,6.000000,7.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,-0.503906,...,4.066299,3.318454,4.015701,3.819806,4.569919,4.062215,4.326886,3.571108,4.204051,4.055685
max,7.000000,8.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,-0.501348,...,6.652909,5.811396,6.271825,5.910567,7.937778,6.950999,6.918053,6.204740,7.152444,6.781264


In [9]:
isFilter = False
if isFilter:
    dpOldErr = get_filtered_errors(coupling_type)
    XY_Train_Filtered = get_filtered_xy(XY_train, dpOldErr)
else:
    XY_Train_Filtered = XY_train    
X_train, y_train = build_x_y_data(XY_Train_Filtered)    
    


In [10]:
#X_data, y_data = build_x_y_data(XY_Filtered)
#X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.3, random_state=228)

In [16]:
%time 
categorical_features = [col for col in X_train if col.startswith('atom_')]

#X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.3, random_state=228)
# to record eval results for plotting
model = LGBMRegressor(**Config.LGB_PARAMS, n_estimators=10000, n_jobs = 4)
model.fit(X_train, y_train, 
        eval_set=[(X_train, y_train), (X_val, y_val)], eval_metric='mae',
        verbose=100, early_stopping_rounds=1000,
         categorical_feature = categorical_features)

y_pred = model.predict(X_val)
print(mean_absolute_error(y_val, y_pred))
np.log(mean_absolute_error(y_val, y_pred))


Wall time: 0 ns


C:\ProgramFiles\Anaconda\lib\site-packages\lightgbm\basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['atom_10', 'atom_2', 'atom_3', 'atom_4', 'atom_5', 'atom_6', 'atom_7', 'atom_8', 'atom_9']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 1000 rounds.
[100]	training's l1: 0.541458	valid_1's l1: 0.597718
[200]	training's l1: 0.419288	valid_1's l1: 0.497061
[300]	training's l1: 0.356466	valid_1's l1: 0.452111
[400]	training's l1: 0.315023	valid_1's l1: 0.424621
[500]	training's l1: 0.284065	valid_1's l1: 0.405936
[600]	training's l1: 0.258355	valid_1's l1: 0.392395
[700]	training's l1: 0.2375	valid_1's l1: 0.382634
[800]	training's l1: 0.219581	valid_1's l1: 0.373591
[900]	training's l1: 0.204425	valid_1's l1: 0.367132
[1000]	training's l1: 0.190725	valid_1's l1: 0.360738
[1100]	training's l1: 0.178722	valid_1's l1: 0.355818
[1200]	training's l1: 0.168122	valid_1's l1: 0.351853
[1300]	training's l1: 0.158525	valid_1's l1: 0.348964
[1400]	training's l1: 0.149885	valid_1's l1: 0.346051
[1500]	training's l1: 0.141971	valid_1's l1: 0.34332
[1600]	training's l1: 0.134428	valid_1's l1: 0.340598
[1700]	training's l1: 0.127874	valid_1's l1: 0.338793
[1800]	training's l1: 0.121611

-1.1545754253311868

In [ ]:
model.get_params()

In [ ]:
eval_results = model.evals_result_
l1 = eval_results["training"]["l1"]
valid = eval_results["valid_1"]["l1"]

import matplotlib.ticker as tic
fig1, ax1 = plt.subplots()
ax1.plot(l1)
ax1.plot(valid)
#ax1.set_yscale('log')
#ax1.set_yticks([0.1, 0.12, 0.2, 0.3])
plt.ylim(0.2, 0.8)   
ax1.get_yaxis().set_major_formatter(tic.ScalarFormatter())
ax1.legend(['Train','Validation'], loc='upper left')
plt.grid(True)
plt.show()


In [ ]:
dif_valid = pd.Series(valid).diff()

In [ ]:
fig1, ax1 = plt.subplots()
ax1.plot(dif_valid)
plt.ylim(-0.0001, 0.0)   

In [ ]:
d_dif_valid = pd.Series(dif_valid).diff()

In [ ]:
fig1, ax1 = plt.subplots()
ax1.plot(d_dif_valid)
plt.ylim(0.00003, -0.00003)   

In [ ]:
plt.plot(y_val, y_pred- y_val, 'ro')

In [ ]:
import seaborn as sns
cols = list(X_train.columns)
df_importance = pd.DataFrame({'feature': cols, 'importance': model.feature_importances_})
sns.barplot(x="importance", y="feature", data=df_importance.sort_values('importance', ascending=False));

In [ ]:
df_importance.sort_values('importance', ascending=False)

In [ ]:
X_train.describe()

In [ ]:
X_train.columns

In [ ]:
aaa

In [ ]:
df = X_train
#Using Pearson Correlation
plt.figure(figsize=(12,10))
cor = df.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()